# Api Gateway with Cognito Authentication

 ### Create User
 
 **Copy your App client id** from Cognito User Pools console and set the **clientId** variable
 
 **note:** the user will need to be confirmed either by email or through the cognito console.

In [26]:
import json, boto3, requests, datetime

# Client ID goes here
clientId = '16bo69frup511mlkl2b6fmk0bo'

client = boto3.client('cognito-idp')

signup_response = client.sign_up(
    ClientId=clientId,
    Username='paul',
    Password='Pa$$word1',
    UserAttributes=[{'Name':'email','Value':'pj642615@gmail.com'}])

print(signup_response)

{'UserConfirmed': False, 'CodeDeliveryDetails': {'Destination': 'p***@g***.com', 'DeliveryMedium': 'EMAIL', 'AttributeName': 'email'}, 'UserSub': 'ecc70b62-5e57-4360-bcd3-b1821227a3c7', 'ResponseMetadata': {'RequestId': '9f458957-f2b3-49ae-8aad-93bc83290c15', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 27 Aug 2021 03:59:33 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '175', 'connection': 'keep-alive', 'x-amzn-requestid': '9f458957-f2b3-49ae-8aad-93bc83290c15'}, 'RetryAttempts': 0}}


### Login
**note:** Make sure to confirm the user

In [28]:
auth_response = client.initiate_auth(
    AuthFlow='USER_PASSWORD_AUTH',
    AuthParameters={
        'USERNAME' : 'paul',
        'PASSWORD' : 'Pa$$word1'
    },
    ClientId=clientId
)

print(auth_response['AuthenticationResult'].keys())

dict_keys(['AccessToken', 'ExpiresIn', 'TokenType', 'RefreshToken', 'IdToken'])


### Call API with authentication header and token

#### Set the gwid to the gateway id

In [52]:
#Set gateway id
gwid = 'a2g4yx6vu0'

headers = {'Authorization': auth_response['AuthenticationResult']['IdToken']}
x = datetime.datetime.now()
orderid = '150'
accountid = 'p150'
vendorid = 'Coffee One'
orderdate = str(x)
coffeetype = 'Short Black'
coffeesize = 'Small'
unitprice = '4.50'
quantity = '1'

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/order')

requests.post(url,json={'order':{
            'orderid': orderid,
            'accountid': accountid,
            'vendorid': vendorid,
            'orderdate':orderdate,
            'details':{
                'coffeetype': coffeetype,
                'coffeesize': coffeesize,
                'unitprice': unitprice,
                'quantity': quantity
            }
        }
    },headers=headers)

response_get = requests.get(url, params={'orderid':150})

print(response_get.json())

{'Item': {'vendorid': 'Coffee One', 'orderdate': '2021-08-27 14:17:43.383216', 'details': {'coffeesize': 'Small', 'coffeetype': 'Short Black', 'quantity': '1', 'unitprice': '4.50'}, 'accountid': 'p150', 'orderid': '150'}, 'ResponseMetadata': {'RequestId': 'NIBJHNF255BLJ5JRN842DB9IBJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Fri, 27 Aug 2021 04:17:43 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '258', 'connection': 'keep-alive', 'x-amzn-requestid': 'NIBJHNF255BLJ5JRN842DB9IBJVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1906745638'}, 'RetryAttempts': 0}}
